In [1]:
import sys
sys.path.insert(0, r'../../libraries/quetzal')
sys.path.insert(0, r'../../libraries/syspy')

from quetzal.io import  gtfs_importer
from quetzal.model import stepmodel, model

import geopandas as gpd

In [2]:
training_folder = '../../'
input_folder = training_folder + r'gis/inputs/'

In [6]:
sm = stepmodel.read_hdf(training_folder + 'model/preparation/base.hdf')

read_hdf: : 100%|██████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 53.78it/s]


In [5]:
sm = stepmodel.read_hdf(training_folder + 'model/model1_assigned_with_road_speed.hdf')

read_hdf: : 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  7.68it/s]


In [7]:
sm.integrity_test_all(errors='ignore')

failed: integrity_test_circular_lines
passed: integrity_test_collision


14731it [00:01, 10289.42it/s]


failed: integrity_test_dead_ends
failed: integrity_test_isolated_roads
passed: integrity_test_nodeset_consistency
failed: integrity_test_road_network
passed: integrity_test_sequences


In [8]:
import networkx as nx

In [9]:
g = nx.Graph()

In [11]:
g.add_edges_from(sm.road_links[['a', 'b']].values.tolist())

In [12]:
l = list(nx.connected_component_subgraphs(g))

In [13]:
len(l)

11

In [18]:
[len(l[i].nodes) for i in range(11)]

[14708, 2, 4, 2, 2, 2, 2, 2, 2, 2, 3]

# add road

In [8]:
road_links = gpd.read_file(input_folder+'road_links.geojson', driver='GeoJSON')
road_nodes = gpd.read_file(input_folder+'road_nodes.geojson', driver='GeoJSON')
road_nodes.set_index('osmid', inplace=True)

sm.road_links = road_links
sm.road_nodes = road_nodes

In [9]:
sm = sm.change_epsg(2154, 'meter')

Reprojecting model from epsg 4326 to epsg 2154: 100%|██████████████████████████████████| 28/28 [00:01<00:00, 17.56it/s]


# road integrity

In [14]:
sm.integrity_fix_collision()

can not add prefixes on table:  footpaths


## road_fix

In [16]:
sm.integrity_fix_road_network(cutoff=10, recursive_depth=5)

direct: 10147it [00:04, 2501.24it/s]
reversed: 10147it [00:03, 2599.54it/s]
9989it [00:03, 2595.75it/s]
direct: 9989it [00:03, 2502.49it/s]
reversed: 9989it [00:03, 2530.48it/s]
9987it [00:03, 2499.50it/s]


## road_test

In [17]:
sm.integrity_test_dead_ends()

9987it [00:00, 11229.43it/s]


In [18]:
sm.integrity_fix_nodeset_consistency()

In [19]:
sm.integrity_test_isolated_roads()

# Transport Integrity

## transport_fix

In [20]:
sm.integrity_fix_collision()
sm.integrity_fix_circular_lines()
sm.integrity_fix_sequences()

dropped broken sequences: ['8226999791364557', '7026391111663108', '126356011531123', '1026353381829393', '6026797241770434', '1026618981581054', '6026797241770433', '1026999531904138', '1026618641034640', '7126352601082481', '1026353381829392', '7826999790881055', '11526352000884475', '1026572650941049', '1026572651003316']


## transport_test

In [21]:
sm.integrity_test_circular_lines()
sm.integrity_test_collision()
sm.integrity_test_nodeset_consistency()
sm.integrity_test_sequences()

In [22]:
sm.integrity_test_all(errors='raise')

passed: integrity_test_circular_lines
passed: integrity_test_collision


9987it [00:00, 10644.04it/s]


passed: integrity_test_dead_ends
passed: integrity_test_isolated_roads
passed: integrity_test_nodeset_consistency


9987it [00:03, 2559.24it/s]


passed: integrity_test_road_network
passed: integrity_test_sequences


# to_file

In [26]:
sm.to_hdf(training_folder + 'model/preparation/road.hdf')   

to_hdf(overwriting): 100%|█████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 14.63it/s]


could not save attribute: missing_road_nodes


In [25]:
sm.missing_nodes = list(sm.missing_nodes)

In [ ]:
s